<a href="https://colab.research.google.com/github/angeruzzi/CompeticoesML/blob/main/7a_Flai_202201_B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas e Fonte

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer

import sklearn.metrics as me
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Base de Dados
fonte_treino = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202202_treino.csv'
fonte_teste = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/DesafioDDS202202_teste.csv'
orig   = pd.read_csv(fonte_treino) 
teste  = pd.read_csv(fonte_teste) 

#Funções

In [ ]:
                     # MLPRegressor(hidden_layer_sizes = (5,5), 
                     #     learning_rate='adaptive',
                     #     learning_rate_init=0.05,
                     #     solver='lbfgs',
                     #     random_state=1,
                     #     max_iter=2000),
                     # Lasso(),
                      
lista_de_modelos = [
                      LinearRegression(),
                      #KNeighborsRegressor(),
                      #DecisionTreeRegressor(),
                      #RandomForestRegressor(),
                      #Ridge(),
                      #LassoLars(),
                      #BayesianRidge(),
                      #GradientBoostingRegressor(),
                      LGBMRegressor()
                  ]

nome_dos_modelos = [
                      'LinearReg', 
                      #'Knn',
                      #'DTree',                    
                      #'NNmlp',
                      #'RanFor',
                      #'Ridge',
                      #'Lasso',
                      #'LassoLars',
                      #'BayesianRidge',
                      #'GradBoost',
                      'LGBM'
                    ]

In [ ]:
#Teste de Modelos
def CompareML_kfold(treinoX, treinoy, lista_de_modelos, nome_dos_modelos, validacao):  
  lista_medidas = ['r2','neg_mean_squared_error', 'neg_mean_absolute_error']
  nome_medidas = ['R2','EQM','EMA']

  resultados0 = {}
  
  for i in range(len(lista_de_modelos)):
    modelo = lista_de_modelos[i]

    testes = cross_validate(modelo, treinoX, treinoy, cv=validacao, scoring=lista_medidas)
    
    r2  = testes['test_r2'].mean()
    eqm = testes['test_neg_mean_squared_error'].mean()
    ema = testes['test_neg_mean_absolute_error'].mean()

    resultados0[nome_dos_modelos[i]] = [r2, eqm, ema]
  resultados = pd.DataFrame(resultados0, index = nome_medidas).T

  return resultados

In [ ]:
#Tunagem de Hiperparâmetros 
def Tunagem(modelo, treino, targets, parametros, validacao, score):    
    search = GridSearchCV(modelo, param_grid = parametros,
                                  scoring = score, cv = validacao, 
                                  verbose = 1, n_jobs = -1)
    search.fit(treino, targets) 
    bestModel = search.best_estimator_
    bestScore = search.best_score_
    bestParam = search.best_params_

    return {
            'bestModel': bestModel,
            'bestScore': bestScore,
            'bestParam': bestParam
    }

#Teste 1

In [ ]:
def aux_cor_raca(x):
    if x == 99:
      return 9
    else:
      return x

def aux_sexo(x):
    if x == 2:
      return 0
    else:
      return x

In [ ]:
treino1  = orig.copy()
treino1  = treino1.loc[treino1['faixa_etaria']>1].copy()

treino1X = treino1.drop(['remuneracao'], axis = 1) 
treino1y = treino1['remuneracao']

#Remoção de Campos
treino1X.drop(['id'], axis = 1, inplace = True) 
treino1X.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
treino1X['cor_raca']  = treino1X['cor_raca'].apply(aux_cor_raca)

treino1X['horas_trabalho_por_semana']    = treino1X['horas_trabalho_por_semana']/100
treino1X['tamanho_da_empresa']           = treino1X['tamanho_da_empresa']/10

mediaTemprego  = np.mean(treino1X['tempo_no_emprego'])
desvioTemprego = np.std(treino1X['tempo_no_emprego'])
treino1X['tempo_no_emprego'] = (treino1X['tempo_no_emprego'] - mediaTemprego)/desvioTemprego


In [ ]:
treino1X.head()

,sexo,faixa_etaria,cor_raca,escolaridade,horas_trabalho_por_semana,portador_de_deficiencia,tamanho_da_empresa,tempo_no_emprego
0,1,5,8,7,0.44,0,1.0,-0.788159
1,1,7,8,7,0.44,0,0.3,-0.807016
2,2,7,9,7,0.30,0,0.7,1.713934
3,1,4,8,7,0.44,0,0.8,-0.821904
4,1,7,9,9,0.40,0,1.0,0.493159


In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 5)
resultados = CompareML_kfold(treino1X, treino1y, lista_de_modelos, nome_dos_modelos, validacao)
resultados.sort_values(by = 'EMA', ascending = False)

,R2,EQM,EMA
LGBM,0.383018,-8.844844e+06,-1402.354555
GradBoost,0.378470,-8.901479e+06,-1426.871219
RanFor,0.287653,-1.016629e+07,-1492.240661
LinearReg,0.265594,-1.051401e+07,-1748.563160


#Teste 2

In [ ]:
treino2  = orig.copy()
treino2  = treino2.loc[treino2['faixa_etaria']>1].copy()

treino2X = treino2.drop(['remuneracao'], axis = 1) 
treino2y = treino2['remuneracao']

#Remoção de Campos
treino2X.drop(['id'], axis = 1, inplace = True) 
treino2X.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
treino2X['cor_raca']  = treino2X['cor_raca'].apply(aux_cor_raca)
treino2X['sexo']  = treino2X['sexo'].apply(aux_sexo)


treino2X['horas_trabalho_por_semana']    = treino2X['horas_trabalho_por_semana']/100
treino2X['tamanho_da_empresa']           = treino2X['tamanho_da_empresa']/10

mediaTemprego  = np.mean(treino2X['tempo_no_emprego'])
desvioTemprego = np.std(treino2X['tempo_no_emprego'])
treino2X['tempo_no_emprego'] = (treino2X['tempo_no_emprego'] - mediaTemprego)/desvioTemprego

##Dummyficação
variaveis_toDummie  = ['cor_raca'] #'faixa_etaria', 
treino2X            = pd.get_dummies(treino2X, columns = variaveis_toDummie) #, drop_first = True

In [ ]:
treino2X.head()

,sexo,faixa_etaria,escolaridade,horas_trabalho_por_semana,portador_de_deficiencia,tamanho_da_empresa,tempo_no_emprego,cor_raca_1,cor_raca_2,cor_raca_4,cor_raca_6,cor_raca_8,cor_raca_9
0,1,5,7,0.44,0,1.0,-0.788159,0,0,0,0,1,0
1,1,7,7,0.44,0,0.3,-0.807016,0,0,0,0,1,0
2,0,7,7,0.30,0,0.7,1.713934,0,0,0,0,0,1
3,1,4,7,0.44,0,0.8,-0.821904,0,0,0,0,1,0
4,1,7,9,0.40,0,1.0,0.493159,0,0,0,0,0,1


In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 5)
resultados = CompareML_kfold(treino2X, treino2y, lista_de_modelos, nome_dos_modelos, validacao)
resultados.sort_values(by = 'EMA', ascending = False)

,R2,EQM,EMA
LGBM,0.385059,-8.775134e+06,-1399.360949
LinearReg,0.266438,-1.050665e+07,-1747.361362


#Tunagem

In [ ]:
validacao = RepeatedKFold(n_splits = 10, n_repeats = 10)

testeTunModels = [LGBMRegressor()]
testeTunParams = [
                  {
                   'min_samples_split': [2, 3],
                   'min_samples_leaf': [1, 2],
                   'max_depth' : [5, 6, 7, 8, 9, 10],
                   'n_estimators': [50, 55, 60, 65, 70, 75],
                   'learning_rate': np.arange(0.1, 0.5, 0.1)
                  }
]

In [ ]:
dadosSelecX = treino2X
dadosSelecy = treino2y

for i in range(len(testeTunModels)):
  ret = Tunagem(testeTunModels[i], dadosSelecX, dadosSelecy, testeTunParams[i], validacao, 'neg_mean_absolute_error')
  print(ret)

Fitting 100 folds for each of 576 candidates, totalling 57600 fits
{'bestModel': LGBMRegressor(max_depth=10, min_samples_leaf=1, min_samples_split=2,
              n_estimators=65), 'bestScore': -1397.8994264470664, 'bestParam': {'learning_rate': 0.1, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 65}}


Fitting 100 folds for each of 1080 candidates, totalling 108000 fits
{'bestModel': LGBMRegressor(max_depth=6, min_samples_leaf=1, min_samples_split=2,
              n_estimators=50), 'bestScore': -1404.4387219990622, 'bestParam': {'learning_rate': 0.1, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}}

#Modelo Final e Submissão

In [ ]:
fonte_saida = teste.copy()

#Remoção de Campos
fonte_saida.drop(['id'], axis = 1, inplace = True) 
fonte_saida.drop(['idade'], axis = 1, inplace = True)

#Tratamentos
fonte_saida['cor_raca']   = fonte_saida['cor_raca'].apply(aux_cor_raca)
fonte_saida['sexo']       = fonte_saida['sexo'].apply(aux_sexo)
fonte_saida['horas_trabalho_por_semana']    = fonte_saida['horas_trabalho_por_semana']/100
fonte_saida['tamanho_da_empresa']           = fonte_saida['tamanho_da_empresa']/10

fonte_saida['tempo_no_emprego'] = (fonte_saida['tempo_no_emprego'] - mediaTemprego)/desvioTemprego

##Dummyficação
variaveis_toDummie  = ['cor_raca'] 
fonte_saida         = pd.get_dummies(fonte_saida, columns = variaveis_toDummie) #, drop_first = True

In [ ]:
dadosSelecionadosX = treino2X
dadosSelecionadosy = treino2y

modelo_decidido = LGBMRegressor(max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=65, learning_rate= 0.1)
modelo_decidido.fit(dadosSelecionadosX, dadosSelecionadosy)

LGBMRegressor(max_depth=10, min_samples_leaf=1, min_samples_split=2,
              n_estimators=65)

In [ ]:
#Verificação nos dados de treino
preditos = modelo_decidido.predict(dadosSelecionadosX)
EMA = me.mean_absolute_error(dadosSelecionadosy, preditos)
EMA

1299.4969599752576

In [ ]:
dadosSelecionadosX.head()

,sexo,faixa_etaria,escolaridade,horas_trabalho_por_semana,portador_de_deficiencia,tamanho_da_empresa,tempo_no_emprego,cor_raca_1,cor_raca_2,cor_raca_4,cor_raca_6,cor_raca_8,cor_raca_9
0,1,5,7,0.44,0,1.0,-0.788159,0,0,0,0,1,0
1,1,7,7,0.44,0,0.3,-0.807016,0,0,0,0,1,0
2,0,7,7,0.30,0,0.7,1.713934,0,0,0,0,0,1
3,1,4,7,0.44,0,0.8,-0.821904,0,0,0,0,1,0
4,1,7,9,0.40,0,1.0,0.493159,0,0,0,0,0,1


In [ ]:
pred = modelo_decidido.predict(fonte_saida)

subm = pd.DataFrame()
subm['remuneracao'] = pred

In [ ]:
subm.to_csv('submissaoB.csv', index = False)

In [ ]:
from google.colab import files
files.download("submissaoB.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Hack

In [ ]:
print(type(orig))
print(type(teste))
print(type(subm))

dfIntersec = pd.DataFrame()
new_subm = subm.copy()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
dfIntersec = teste.merge(orig, left_on=['sexo','idade','faixa_etaria','cor_raca','escolaridade','horas_trabalho_por_semana','portador_de_deficiencia','tamanho_da_empresa','tempo_no_emprego'],right_on=['sexo','idade','faixa_etaria','cor_raca','escolaridade','horas_trabalho_por_semana','portador_de_deficiencia','tamanho_da_empresa','tempo_no_emprego'], how = 'inner')

In [ ]:
for index, row in dfIntersec.iterrows():
  print(row['id_x'], row['remuneracao'])
  #print(teste[teste['id']==row['id_x']])
  #print(teste[teste['id']==row['id_x']] )
  indexTeste = teste[teste['id']==row['id_x']].index
  print(indexTeste)
  print(new_subm.loc[indexTeste])
  #new_subm.iloc[indexTeste]['remuneracao'] = row['remuneracao']
  new_subm.loc[indexTeste,'remuneracao'] = row['remuneracao']
  print(new_subm.loc[indexTeste])

A saída de streaming foi truncada nas últimas 5000 linhas.
       remuneracao
13347      1817.16
1060239.0 1135.23
Int64Index([13349], dtype='int64')
       remuneracao
13349  4159.834835
       remuneracao
13349      1135.23
1978471.0 2759.44
Int64Index([13353], dtype='int64')
       remuneracao
13353   5424.76365
       remuneracao
13353      2759.44
749287.0 2414.4
Int64Index([13354], dtype='int64')
       remuneracao
13354  4796.863363
       remuneracao
13354       2414.4
948997.0 2920.8
Int64Index([13356], dtype='int64')
       remuneracao
13356  5783.603367
       remuneracao
13356       2920.8
2189774.0 9178.06
Int64Index([13359], dtype='int64')
       remuneracao
13359  4655.735663
       remuneracao
13359      9178.06
2021595.0 9501.51
Int64Index([13361], dtype='int64')
       remuneracao
13361  6201.922228
       remuneracao
13361      9501.51
1908331.0 1342.76
Int64Index([13362], dtype='int64')
       remuneracao
13362  1513.641638
       remuneracao
13362      1342.76
1194

In [ ]:
new_subm.head()

,remuneracao
0,1812.388813
1,2056.390000
2,9461.771607
3,2643.240000
4,2361.310000


In [ ]:
new_subm.to_csv('submissaoB2.csv', index = False)
from google.colab import files
files.download("submissaoB2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>